<a href="https://colab.research.google.com/github/WestHamster/100DAYSOFML/blob/master/Day-3(Kaggle-quora_lstm).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
     !pip install -U -q PyDrive ## you will have install for every colab session

     from pydrive.auth import GoogleAuth
     from pydrive.drive import GoogleDrive
     from google.colab import auth
     from oauth2client.client import GoogleCredentials

     # 1. Authenticate and create the PyDrive client.
     auth.authenticate_user()
     gauth = GoogleAuth()
     gauth.credentials = GoogleCredentials.get_application_default()
     drive = GoogleDrive(gauth)


In [0]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
ls

adc.json  drive/  sample_data/


In [0]:
import os
from google.colab import files

os.chdir("drive/My Drive/TCS/Quora_q")

In [0]:
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation,Lambda
from keras.layers.merge import concatenate
from keras.models import Model,Sequential
from keras import backend as K
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import confusion_matrix as cf

import sys
stdout = sys.stdout


Using TensorFlow backend.


In [0]:
#BASE_DIR = 'C:\\videoken\\walmart\\assignment5/week7_assignment_data/'
TRAIN_DATA_FILE = 'train.csv'
TEST_DATA_FILE = 'test.csv'



In [0]:
# parameters
MAX_SEQUENCE_LENGTH = 100
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [0]:
def prepare_pad_sequences(data_col):
    sequences = tokenizer.texts_to_sequences(data_col)
    d_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(d_index))
    data_seq = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    return data_seq


In [0]:
!pip install gensim
!pip install nltk

In [0]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /root/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Package biocreative_ppi is already up-to-date!
       | Downloading package brown to /root/nltk_data...
       |   Package brown is already up-to-date!
       | Downloading package brown_tei to /root/nltk_data...
       |   Package brown_tei is already up-to-date!
       | Downloading package cess_cat to /root/nltk_data...
       |   Package cess_cat is already up-to-date!
       | Down

True

In [0]:
df = pd.read_csv('questions.csv',usecols=['question1','question2','is_duplicate'])

## Read data 

In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test_withlabels.csv')
texts = train['question1'].tolist()
texts.extend(train['question2'].tolist())


tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

print('Shape of data tensor:', data.shape)

print(train['question1'].shape)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 15579 unique tokens.
Shape of data tensor: (20000, 100)
(10000,)


In [0]:
train_q1 = train['question1'].tolist()
train_q2 = train['question2'].tolist()

x_train_1 = prepare_pad_sequences(train_q1)
x_train_2 = prepare_pad_sequences(train_q2)

train_label = train['is_duplicate']

test_q1 = test['question1'].tolist()
test_q2 = test['question2'].tolist()

x_test_1 = prepare_pad_sequences(test_q1)
x_test_2 = prepare_pad_sequences(test_q2)

test_label = test['is_duplicate']

texts = x_train_1
#pd.concat(texts,x_train_2)
print(x_train_1)


Found 15579 unique tokens.
Found 15579 unique tokens.
Found 15579 unique tokens.
Found 15579 unique tokens.
[[   0    0    0 ...  449    8   36]
 [   0    0    0 ...    4 6070 3988]
 [   0    0    0 ...  157    7 3064]
 ...
 [   0    0    0 ...  847 1249  653]
 [   0    0    0 ...   40   14 8705]
 [   0    0    0 ...  392 1562  919]]


In [0]:
import tensorflow as tf
"""config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))
"""
from keras.layers import Bidirectional, GlobalMaxPool1D, concatenate, dot
max_features=3000
embedding_size = 100
inp1 = Input(shape=(100,))
inp2 = Input(shape=(100,))

x1 = Embedding(max_features, embedding_size)(inp1)
x2 = Embedding(max_features, embedding_size)(inp2)

x3 = LSTM(32, return_sequences = True)(x1)
x4 = LSTM(32, return_sequences = True)(x2)

x5 = GlobalMaxPool1D()(x3)
x6 = GlobalMaxPool1D()(x4)

x7 = dot([x5, x6], axes=1)

x8 = Dense(40, activation='relu')(x7)
x9 = Dropout(0.05)(x8)
x10 = Dense(10, activation='relu')(x9)
output = Dense(1, activation='sigmoid')(x10)

model = Model(inputs=[inp1, inp2], outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
batch_size = 8
epochs = 5

#model.fit([x_train_1, x_train_2], train_label, batch_size=batch_size, epochs=epochs, validation_split=0.2)



In [0]:
import keras
from keras.models import load_model

model=keras.models.load_model('quora.h5')
model.fit([x_train_1, x_train_2], train_label, batch_size=8, epochs=5, validation_split=0.2)

#x_mode = model.predict([x_train_1, x_train_2])


Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 546s 68ms/step - loss: 0.6628 - acc: 0.6266 - val_loss: 0.6570 - val_acc: 0.6340
Epoch 2/5
8000/8000 [==============================] - 542s 68ms/step - loss: 0.6603 - acc: 0.6276 - val_loss: 0.6572 - val_acc: 0.6340
Epoch 3/5
8000/8000 [==============================] - 547s 68ms/step - loss: 0.6606 - acc: 0.6276 - val_loss: 0.6570 - val_acc: 0.6340
Epoch 4/5
8000/8000 [==============================] - 557s 70ms/step - loss: 0.6605 - acc: 0.6276 - val_loss: 0.6569 - val_acc: 0.6340
Epoch 5/5
8000/8000 [==============================] - 548s 69ms/step - loss: 0.6605 - acc: 0.6276 - val_loss: 0.6569 - val_acc: 0.6340


In [0]:
x_mode=(x_mode>0.5)+[0]

In [0]:
from sklearn.metrics import accuracy_score

accuracy_score(train_label,x_mode)

0.6169

# **SEQUENTIAL**

In [0]:
from keras.models import Sequential
from keras import layers
import warnings

warnings.filterwarnings('ignore')
inp1 = Input(shape=(100,))
inp2 = Input(shape=(100,))

x1 = Embedding(max_features, embedding_size)(inp1)
x2 = Embedding(max_features, embedding_size)(inp2)


input_dim = x_train_1.shape[1]

model = Sequential()
model.add(layers.Dense(15,input_dim=input_dim,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [0]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [0]:
x_test_1.shape

In [0]:
model_1 = model.fit(x_test_1,train_label,epochs=35,verbose=False,validation_data=(x_test_1,test_label),batch_size=130)

In [0]:
loss,accuracy = model.evaluate(x_train_1,train_label,verbose=False)
print("Training accuracy:",accuracy)
loss,accuracy = model.evaluate(x_test_1,test_label,verbose=False)
print("Test accuracy:",accuracy)
